INDICACIONES INICIALES:

- Correr en GPU
- Las carpetas deben subirse en zip
- Si se entrena el entorno, debe descargarse best. pt usando bloque 17, luego volver a subirlo usando bloque 3

#
$$\textbf{Entrenamiento del modelo}$$

##
$$\textbf{Bloque 1 — Preparación del entorno:}
\\ \text{Crea la estructura de carpetas del proyecto en Colab (inputs, temporales, dataset, runs y resultados) e instala librerías.}$$


In [9]:
# Importa herramientas para manejar rutas, archivos, comandos y datos
from pathlib import Path                  # Permite trabajar con rutas de carpetas/archivos de forma simple
import shutil                             # Sirve para copiar/mover/borrar archivos y carpetas
import zipfile                            # Sirve para descomprimir/comprimir archivos .zip
import subprocess                         # Sirve para ejecutar comandos del sistema (ffmpeg, etc.)
import os                                 # Utilidades del sistema (rutas, variables, etc.)
import glob                               # Buscar archivos con patrones (ej: *.jpg)
import re                                 # Manejo de texto con patrones (regex)
import pandas as pd                       # Para crear/guardar tablas (CSV)

BASE = Path("/content")                   # Define la carpeta raíz del entorno Colab
DIR_IN = BASE/"in"                        # Carpeta para archivos que tú subes (video/zip)
DIR_WORK = BASE/"work"                    # Carpeta para trabajo temporal (frames, raw, etc.)
DIR_DATASET = BASE/"dataset"              # Carpeta del dataset final en formato YOLO
DIR_RUNS = BASE/"runs"                    # Carpeta donde YOLO guarda entrenamientos/predicciones
DIR_OUT = BASE/"out"                      # Carpeta para resultados finales listos para descargar

for d in [DIR_IN, DIR_WORK, DIR_DATASET, DIR_RUNS, DIR_OUT]:  # Recorre cada carpeta necesaria
    d.mkdir(parents=True, exist_ok=True)                      # Crea la carpeta si no existe

print("✅ Carpetas listas:")              # Muestra confirmación
print("IN:", DIR_IN)                     # Imprime ruta de inputs
print("WORK:", DIR_WORK)                 # Imprime ruta de temporales
print("DATASET:", DIR_DATASET)           # Imprime ruta del dataset final
print("RUNS:", DIR_RUNS)                 # Imprime ruta de salidas de YOLO
print("OUT:", DIR_OUT)                   # Imprime ruta de resultados descargables

# Instala la librería Ultralytics (YOLO) en el entorno de Colab
!pip -q install ultralytics

# Importa YOLO para confirmar que la instalación quedó OK
from ultralytics import YOLO

# Muestra la versión de FFmpeg para verificar que está instalado
!ffmpeg -version | head -n 2

# Lista la GPU disponible (si hay) para confirmar aceleración por hardware
!nvidia-smi -L || true

# Imprime confirmación de que Ultralytics quedó listo
print("✅ Ultralytics (YOLO) instalado y disponible")


✅ Carpetas listas:
IN: /content/in
WORK: /content/work
DATASET: /content/dataset
RUNS: /content/runs
OUT: /content/out
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
/bin/bash: line 1: nvidia-smi: command not found
✅ Ultralytics (YOLO) instalado y disponible


##
$$\textbf{Bloque 4 — Exploración y control de espacio:} \
\\\ \text{Te muestra qué hay dentro de las carpetas del proyecto  y deja listas las carpetas de trabajo para frames y dataset crudo.}$$


In [13]:
# Define una función simple para listar contenido de una carpeta sin tirar error
def listar_carpeta(ruta, max_items=50):                  # Crea una función para listar archivos/carpetas
    ruta = Path(ruta)                                    # Convierte la ruta a formato Path
    print("\n📁", ruta)                                   # Imprime la carpeta que se está revisando
    if not ruta.exists():                                # Si la carpeta no existe
        print("⚠️ No existe todavía.")                    # Avisa sin romper el notebook
        return                                            # Termina la función

    items = sorted(list(ruta.iterdir()), key=lambda p: (p.is_file(), p.name.lower()))  # Ordena primero carpetas
    if len(items) == 0:                                  # Si no hay nada dentro
        print("⚠️ Está vacía.")                           # Avisa sin romper el notebook
        return                                            # Termina la función

    for p in items[:max_items]:                          # Recorre hasta max_items elementos
        tipo = "DIR " if p.is_dir() else "FILE"          # Marca si es carpeta o archivo
        print(f" - {tipo} {p.name}")                     # Imprime el nombre

    if len(items) > max_items:                           # Si hay más de max_items
        print(f" ... y {len(items)-max_items} más")      # Avisa cuántos faltan por mostrar

# Lista las carpetas principales del proyecto para ver el estado general
print("✅ Listado rápido de carpetas del proyecto:")      # Mensaje de inicio
listar_carpeta("/content")                               # Muestra lo que hay en /content
listar_carpeta(DIR_IN)                                   # Muestra lo que hay en /content/in
listar_carpeta(DIR_WORK)                                 # Muestra lo que hay en /content/work
listar_carpeta(DIR_DATASET)                              # Muestra lo que hay en /content/dataset
listar_carpeta(DIR_RUNS)                                 # Muestra lo que hay en /content/runs
listar_carpeta(DIR_OUT)                                  # Muestra lo que hay en /content/out

# Muestra el espacio total del disco del entorno (si el comando está disponible)
print("\n✅ Espacio total del entorno (df -h):")           # Título del chequeo
try:
    subprocess.run(["df", "-h"], check=False)            # Ejecuta df sin romper si falla
except Exception:
    print("⚠️ No pude ejecutar df -h en este entorno.")   # Aviso si no se pudo

# Muestra cuánto pesa cada carpeta dentro de /content (para detectar qué está llenando)
print("\n✅ Peso por carpeta dentro de /content (du -h):") # Título del chequeo
try:
    subprocess.run(["bash", "-lc", "du -h --max-depth=1 /content | sort -h"], check=False)  # Ejecuta du ordenado
except Exception:
    print("⚠️ No pude ejecutar du en este entorno.")      # Aviso si no se pudo

# Detecta automáticamente qué tipo de archivo tienes en /content/in (sin tirar error si no hay nada)
in_files = list(DIR_IN.iterdir())                               # Lee todo lo que hay dentro de /content/in

# Busca un video por extensión común (si existe)
video = next((p for p in in_files if p.suffix.lower() in [".mp4", ".mov", ".avi", ".mkv"]), None)  # Encuentra el primer video

# Busca un zip que se llame images.zip (si existe)
zip_images = next((p for p in in_files if p.name.lower() == "images.zip"), None)  # Encuentra images.zip

# Busca un zip que parezca export de CVAT (si existe)
zip_cvat = next((p for p in in_files if p.suffix.lower() == ".zip" and "cvat" in p.name.lower()), None)  # Encuentra zip CVAT

# Define carpetas de trabajo estándar (frames y dataset_raw)
FRAMES_DIR = DIR_WORK/"frames"                                  # Carpeta donde se guardan frames extraídos del video
RAW_DIR = DIR_WORK/"dataset_raw"                                # Carpeta donde se descomprime el export de CVAT

# Crea las carpetas si no existen
FRAMES_DIR.mkdir(parents=True, exist_ok=True)                   # Crea /content/work/frames si no existe
RAW_DIR.mkdir(parents=True, exist_ok=True)                      # Crea /content/work/dataset_raw si no existe

# Imprime un resumen claro de lo que se detectó
print("✅ Detección de inputs en /content/in:")                  # Título del resumen
print(" - Video:", video.name if video else "No detectado")      # Informa si hay video
print(" - images.zip:", zip_images.name if zip_images else "No detectado")  # Informa si hay zip de imágenes
print(" - zip CVAT:", zip_cvat.name if zip_cvat else "No detectado")        # Informa si hay export CVAT

# Imprime rutas de trabajo listas
print("\n✅ Rutas de trabajo listas:")                           # Título para rutas
print(" - FRAMES_DIR:", FRAMES_DIR)                              # Ruta donde irán los frames
print(" - RAW_DIR:", RAW_DIR)                                    # Ruta donde irá el dataset crudo

# Mensaje guía para tu siguiente decisión (sin obligarte a nada)
print("\nℹ️ Próximo paso sugerido:")                              # Guía de flujo
if video:
    print(" - Tienes video: puedes extraer frames (Bloque 6).")   # Recomienda ruta A
elif zip_cvat:
    print(" - Tienes export CVAT: puedes descomprimir y normalizar dataset (Bloques 8+).")  # Recomienda ruta CVAT
elif zip_images:
    print(" - Tienes images.zip: puedes descomprimir para etiquetar (si lo necesitas).")    # Recomienda ruta B
else:
    print(" - Aún no hay inputs: puedes seguir armando notebook igual y subir después.")    # Recomienda seguir sin datos



✅ Listado rápido de carpetas del proyecto:

📁 /content
 - DIR  .config
 - DIR  .ipynb_checkpoints
 - DIR  dataset
 - DIR  in
 - DIR  out
 - DIR  runs
 - DIR  sample_data
 - DIR  train
 - DIR  work
 - FILE data.yaml

📁 /content/in
 - DIR  .ipynb_checkpoints
 - FILE cvat t1p2 t10 y t3p2.zip

📁 /content/work
 - DIR  dataset_raw
 - DIR  frames

📁 /content/dataset
 - DIR  images
 - DIR  masks

📁 /content/runs
 - DIR  predict_val
 - DIR  train_unet

📁 /content/out
 - FILE dataset_summary.txt
 - FILE val_predictions.zip

✅ Espacio total del entorno (df -h):

✅ Peso por carpeta dentro de /content (du -h):
✅ Detección de inputs en /content/in:
 - Video: No detectado
 - images.zip: No detectado
 - zip CVAT: cvat t1p2 t10 y t3p2.zip

✅ Rutas de trabajo listas:
 - FRAMES_DIR: /content/work/frames
 - RAW_DIR: /content/work/dataset_raw

ℹ️ Próximo paso sugerido:
 - Tienes export CVAT: puedes descomprimir y normalizar dataset (Bloques 8+).


##
$$\textbf{Bloque 8 — Importar export de CVAT:} \
\\\ \text{Si hay zip de CVAT (BLOQUE 3), lo descomprime en /content #/work/dataset_raw. Si no tiene cvat revisar bloque 6.}$$


In [14]:
# Descomprime el export de CVAT (si existe) en /content/work/dataset_raw (si no existe, no falla: avisa)
# Nota: Este zip suele llamarse algo como "cvat_yolo_export.zip" (el nombre puede variar)

# Vuelve a buscar un zip que parezca de CVAT por si lo subiste recién
in_files = list(DIR_IN.iterdir())                                              # Lee archivos en /content/in
zip_cvat = next((p for p in in_files if p.suffix.lower() == ".zip" and "cvat" in p.name.lower()), None)  # Busca zip con "cvat" en el nombre

# Si no encontramos zip CVAT, avisamos y seguimos sin error
if zip_cvat is None:                                                           # Revisa si existe export de CVAT
    print("⚠️ No detecté ningún .zip de CVAT en /content/in (debe tener 'cvat' en el nombre).")           # Aviso
    print("ℹ️ Cuando lo tengas, súbelo y vuelve a correr este bloque.")         # Guía
else:
    # Limpia dataset_raw anterior para evitar mezclar versiones
    if RAW_DIR.exists():                                                       # Revisa si ya existe dataset_raw
        shutil.rmtree(RAW_DIR)                                                 # Borra dataset_raw anterior completo
    RAW_DIR.mkdir(parents=True, exist_ok=True)                                 # Crea dataset_raw limpio

    # Descomprime el zip de CVAT dentro de dataset_raw
    with zipfile.ZipFile(zip_cvat, "r") as z:                                  # Abre el zip de CVAT
        z.extractall(RAW_DIR)                                                  # Extrae todo su contenido en RAW_DIR

    # Lista contenido para confirmar que se extrajo algo
    extracted_any = any(RAW_DIR.rglob("*"))                                     # Verifica si hay archivos extraídos
    if not extracted_any:                                                      # Si no se extrajo nada
        print("⚠️ El zip se descomprimió, pero no veo archivos dentro. Revisa si el zip está correcto.")  # Aviso
    else:
        print("✅ Export CVAT descomprimido en:", RAW_DIR)                      # Confirmación
        # Muestra un vistazo rápido de archivos/carpetas extraídas
        top_items = sorted(list(RAW_DIR.iterdir()))                             # Lista el primer nivel de dataset_raw
        print("✅ Primer nivel dentro de dataset_raw:")                         # Título del listado
        for p in top_items[:30]:                                                # Muestra hasta 30 ítems
            tag = "DIR " if p.is_dir() else "FILE"                              # Marca si es carpeta o archivo
            print(" -", tag, p.name)                                            # Imprime nombre
        if len(top_items) > 30:                                                 # Si hay muchos ítems
            print(f" ... y {len(top_items)-30} más")                            # Indica que hay más


✅ Export CVAT descomprimido en: /content/work/dataset_raw
✅ Primer nivel dentro de dataset_raw:
 - DIR  cvat t1p2 t10 y t3p2


##
$$\textbf{Bloque 9 — Detectar (CVAT) + Normalizar YOLO + Split:} \
\\\ \text{Busca automáticamente imágenes y labels dentro de #dataset\_raw, y construye el split en #/content/dataset.}$$


In [15]:
# Detecta imágenes/labels en dataset_raw y construye dataset YOLO final (train/val) creando .txt vacíos para negativos
import random                                                    # Sirve para mezclar antes del split

img_exts = {".jpg", ".jpeg", ".png"}                             # Extensiones válidas de imágenes
train_ratio = 0.8                                                # Proporción train/val
seed = 42                                                        # Semilla para split repetible

# Define rutas YOLO estándar de salida
IMG_TRAIN = DIR_DATASET/"images/train"                           # Imágenes train
IMG_VAL   = DIR_DATASET/"images/val"                             # Imágenes val
LBL_TRAIN = DIR_DATASET/"labels/train"                           # Labels train
LBL_VAL   = DIR_DATASET/"labels/val"                             # Labels val

# Crea carpetas de salida
for d in [IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL]:               # Recorre carpetas YOLO
    d.mkdir(parents=True, exist_ok=True)                         # Crea si falta

# Verifica que exista dataset_raw
if not RAW_DIR.exists():                                         # Si no existe dataset_raw
    print("⚠️ No existe /content/work/dataset_raw todavía. Corre el Bloque 8 (descomprimir CVAT) primero.")  # Aviso
else:
    # Busca imágenes y txt dentro de dataset_raw (recursivo)
    all_imgs = [p for p in RAW_DIR.rglob("*") if p.suffix.lower() in img_exts]   # Todas las imágenes
    all_txt  = [p for p in RAW_DIR.rglob("*.txt")]                               # Todos los txt

    print("✅ Archivos encontrados dentro de dataset_raw:")       # Resumen inicial
    print(" - Imágenes:", len(all_imgs))                         # Cantidad imágenes
    print(" - TXT:", len(all_txt))                               # Cantidad txt

    # Si falta algo, avisa pero no rompe
    if len(all_imgs) == 0:
        print("⚠️ No encontré imágenes dentro de dataset_raw. Revisa estructura del export (o tu zip).")  # Aviso
    if len(all_txt) == 0:
        print("⚠️ No encontré labels (.txt) dentro de dataset_raw. Ojo: CVAT solo exporta txt con anotaciones.")  # Aviso

    # Detecta carpeta con más imágenes y carpeta con más txt (candidatas principales)
    img_dir = None                                               # Carpeta candidata de imágenes
    lbl_dir = None                                               # Carpeta candidata de labels

    if len(all_imgs) > 0:                                        # Si hay imágenes
        counts_img_parent = {}                                   # Conteo por carpeta
        for p in all_imgs:                                       # Recorre imágenes
            counts_img_parent[p.parent] = counts_img_parent.get(p.parent, 0) + 1  # Cuenta
        img_dir = max(counts_img_parent, key=counts_img_parent.get)              # Elige mayor

    if len(all_txt) > 0:                                         # Si hay txt
        counts_lbl_parent = {}                                   # Conteo por carpeta
        for p in all_txt:                                        # Recorre txt
            counts_lbl_parent[p.parent] = counts_lbl_parent.get(p.parent, 0) + 1  # Cuenta
        lbl_dir = max(counts_lbl_parent, key=counts_lbl_parent.get)              # Elige mayor

    print("\n✅ Rutas detectadas (candidatas principales):")      # Imprime rutas
    print(" - img_dir:", str(img_dir) if img_dir else "No detectado")  # Carpeta imágenes
    print(" - lbl_dir:", str(lbl_dir) if lbl_dir else "No detectado")  # Carpeta labels

    # Si no se detectaron rutas, termina sin error
    if (img_dir is None) or (lbl_dir is None):
        print("⚠️ No pude detectar img_dir y/o lbl_dir. Revisa qué hay dentro de dataset_raw (Bloque 4 listar carpetas).")  # Aviso
    else:
        # Crea mapas por stem para hacer match imagen <-> label
        img_map = {}                                             # stem -> ruta imagen
        for p in img_dir.iterdir():                              # Recorre archivos en img_dir
            if p.suffix.lower() in img_exts:                     # Si es imagen
                img_map[p.stem] = p                              # Guarda

        lbl_map = {}                                             # stem -> ruta label
        for p in lbl_dir.iterdir():                              # Recorre archivos en lbl_dir
            if p.suffix.lower() == ".txt":                       # Si es txt
                lbl_map[p.stem] = p                              # Guarda

        # Diagnóstico de matching
        all_stems = sorted(img_map.keys())                       # Todas las imágenes (incluye negativos)
        paired = sorted(set(img_map.keys()) & set(lbl_map.keys()))            # Con label
        missing_lbl = sorted(set(img_map.keys()) - set(lbl_map.keys()))      # Sin label (negativos)
        missing_img = sorted(set(lbl_map.keys()) - set(img_map.keys()))      # Txt sin imagen

        print("\n✅ Matching imagen + label (incluyendo negativos):")  # Reporte matching
        print(" - Total imágenes:", len(all_stems))              # Total
        print(" - Con label (.txt):", len(paired))               # Con txt
        print(" - Sin label (se creará .txt vacío):", len(missing_lbl))  # Negativos
        print(" - Txt sin imagen (se ignoran):", len(missing_img))       # Huérfanos

        # Si no hay imágenes, no se puede construir dataset
        if len(all_stems) == 0:
            print("⚠️ No hay imágenes válidas para construir dataset. Revisa nombres/extensiones.")  # Aviso
        else:
            # Limpia salidas anteriores
            for d in [IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL]:   # Recorre carpetas de salida
                for f in d.glob("*"):                            # Recorre archivos dentro
                    f.unlink()                                   # Borra

            # Split train/val sobre TODAS las imágenes
            random.seed(seed)                                    # Semilla
            random.shuffle(all_stems)                             # Mezcla
            cut = int(len(all_stems) * train_ratio)              # Corte
            train_ids = all_stems[:cut]                          # Train stems
            val_ids = all_stems[cut:]                            # Val stems

            # Copia imagen y label (o crea vacío si falta)
            def copy_img_and_label(stem, img_dst, lbl_dst):      # Función copiar + label
                img_src = img_map[stem]                          # Imagen fuente
                shutil.copy2(img_src, img_dst / img_src.name)    # Copia imagen

                lbl_out = lbl_dst / f"{stem}.txt"                # Label destino
                if stem in lbl_map:                              # Si existe label real
                    shutil.copy2(lbl_map[stem], lbl_out)         # Copia label
                else:
                    lbl_out.write_text("", encoding="utf-8")     # Crea label vacío (negativo)

            # Copia a train
            for s in train_ids:                                  # Recorre train
                copy_img_and_label(s, IMG_TRAIN, LBL_TRAIN)      # Copia

            # Copia a val
            for s in val_ids:                                    # Recorre val
                copy_img_and_label(s, IMG_VAL, LBL_VAL)          # Copia

            # Resumen final
            n_train_img = len(list(IMG_TRAIN.glob("*")))         # Cuenta imgs train
            n_train_lbl = len(list(LBL_TRAIN.glob("*.txt")))     # Cuenta labels train
            n_val_img = len(list(IMG_VAL.glob("*")))             # Cuenta imgs val
            n_val_lbl = len(list(LBL_VAL.glob("*.txt")))         # Cuenta labels val

            print("\n✅ Dataset YOLO final creado en:", DIR_DATASET)  # Confirma creación
            print(" - Train: imgs =", n_train_img, "| lbls =", n_train_lbl)  # Resumen train
            print(" - Val:   imgs =", n_val_img,   "| lbls =", n_val_lbl)    # Resumen val

            # Chequeo 1 txt por imagen (ideal)
            if n_train_img != n_train_lbl:
                print("⚠️ Ojo: TRAIN imgs != txt (deberían ser iguales).")  # Aviso
            if n_val_img != n_val_lbl:
                print("⚠️ Ojo: VAL imgs != txt (deberían ser iguales).")    # Aviso

            # Guarda resumen para trazabilidad
            summary_path = DIR_OUT/"dataset_summary.txt"         # Archivo resumen
            summary_text = (
                f"total_imgs={len(all_stems)}\n"
                f"imgs_with_lbl={len(paired)}\n"
                f"imgs_without_lbl_created_empty={len(missing_lbl)}\n"
                f"txt_without_img_ignored={len(missing_img)}\n"
                f"train_ratio={train_ratio}\n"
                f"train_imgs={n_train_img}\ntrain_lbls={n_train_lbl}\n"
                f"val_imgs={n_val_img}\nval_lbls={n_val_lbl}\n"
            )
            summary_path.write_text(summary_text, encoding="utf-8")  # Escribe resumen
            print("✅ Resumen guardado en:", summary_path)        # Confirma guardado


✅ Archivos encontrados dentro de dataset_raw:
 - Imágenes: 885
 - TXT: 321

✅ Rutas detectadas (candidatas principales):
 - img_dir: /content/work/dataset_raw/cvat t1p2 t10 y t3p2/images
 - lbl_dir: /content/work/dataset_raw/cvat t1p2 t10 y t3p2/labels

✅ Matching imagen + label (incluyendo negativos):
 - Total imágenes: 885
 - Con label (.txt): 321
 - Sin label (se creará .txt vacío): 564
 - Txt sin imagen (se ignoran): 0

✅ Dataset YOLO final creado en: /content/dataset
 - Train: imgs = 708 | lbls = 708
 - Val:   imgs = 177 | lbls = 177
✅ Resumen guardado en: /content/out/dataset_summary.txt


##
$$\textbf{Bloque 11 — Crear data.yaml (receta del dataset):} \
\\\ \text{Genera el archivo data.yaml que le dice a YOLO dónde está tu dataset (train/val) y se crean manualmente las clases.}$$


In [16]:
# Crea el archivo data.yaml para YOLO (si no hay dataset aún, no falla: avisa)
yaml_path = BASE/"data.yaml"                                    # Define la ruta donde se guardará el YAML

# Define aquí tus clases EXACTAS y en el MISMO orden que usaste en CVAT - OJO DEBE HACERSE MANUAL
classes = [
    "falla junta paramento izquierdo",
    "falla junta paramento derecho",
    "falla junta losa fondo",
    "estría lado izquierdo",
    "estría lado derecho",
    "estría centro",
    "daño paramento",
    "daño losa fondo",
]                                                               # Lista de nombres de clases (edítala tú)

# Verifica que existan carpetas train/val para evitar un YAML apuntando a nada
train_dir_ok = (DIR_DATASET/"images/train").exists()            # Revisa si existe la carpeta de imágenes train
val_dir_ok = (DIR_DATASET/"images/val").exists()                # Revisa si existe la carpeta de imágenes val

# Si no hay estructura dataset, avisa y no rompe el notebook
if not (train_dir_ok and val_dir_ok):                           # Si faltan carpetas básicas del dataset
    print("⚠️ No detecto la estructura de dataset en /content/dataset/images/train y /val.")  # Aviso
    print("ℹ️ Corre el Bloque 10 (normalización + split) antes de crear el data.yaml.")       # Guía
else:
    # Si no definiste clases, avisa para que no entrenes con un YAML incompleto
    if len(classes) == 0:                                       # Si la lista de clases está vacía
        print("⚠️ La lista 'classes' está vacía. Agrega tus clases en el orden de CVAT antes de entrenar.")  # Aviso
        print("ℹ️ Igual crearé el data.yaml, pero NO deberías entrenar hasta completar 'classes'.")          # Guía

    # Construye el contenido del YAML que YOLO necesita
    yaml_text = f"""path: {DIR_DATASET}
train: images/train
val: images/val
names:
"""                                                             # Texto base del YAML (path + rutas train/val)

    # Agrega las clases con su índice (0,1,2...) en el orden correcto
    for i, c in enumerate(classes):                              # Recorre clases con índice
        yaml_text += f"  {i}: {c}\n"                              # Agrega cada clase al YAML

    # Guarda el archivo data.yaml
    yaml_path.write_text(yaml_text, encoding="utf-8")            # Escribe el YAML en disco

    # Prints de confirmación + vista rápida del contenido
    print("✅ data.yaml creado en:", yaml_path)                   # Confirma ruta del archivo creado
    print("✅ Contenido de data.yaml:")                           # Título del contenido
    print(yaml_text)                                              # Muestra el texto completo


✅ data.yaml creado en: /content/data.yaml
✅ Contenido de data.yaml:
path: /content/dataset
train: images/train
val: images/val
names:
  0: falla junta paramento izquierdo
  1: falla junta paramento derecho
  2: falla junta losa fondo
  3: estría lado izquierdo
  4: estría lado derecho
  5: estría centro
  6: daño paramento
  7: daño losa fondo



##
$$\textbf{Bloque 12 — Entrenamiento YOLO:} \
\\\ \text{Entrena un modelo YOLO usando data.yaml y guarda los pesos (best.pt/last.pt) y métricas dentro de #/content/runs/train.}$$


In [ ]:
# Bloque 12 — Entrenamiento YOLO (Optimizado para aprendizaje continuo)

from ultralytics import YOLO
import shutil

# 1. Configuración de parámetros
data_yaml = "/content/data.yaml"
batch_size = 16  # Si Colab te da error de "Out of Memory", baja esto a 8 o 4, si no, mantén en 16
epochs = 50
img_size = 640
output_dir = "/content/runs" # Ajustado para que Ultralytics maneje la estructura interna
best_pt_path = "/content/in/best.pt"

# 2. Lógica de carga de modelo (Transfer Learning)
if Path(best_pt_path).exists():
    print(f"✅ Conocimiento previo detectado. Cargando: {best_pt_path}")
    model = YOLO(best_pt_path)
    # Recomendación: Si retomas, bajamos un poco la tasa de aprendizaje inicial (lr0)
    # para que no olvide lo anterior bruscamente.
    lr_inicial = 0.001
else:
    print("ℹ️ Iniciando desde cero con YOLOv8 Nano.")
    model = YOLO("yolov8n.pt")
    lr_inicial = 0.01 # Tasa estándar para modelos nuevos

# 3. Entrenamiento
print("🚀 Iniciando entrenamiento con el dataset de 1300 imágenes...")
results = model.train(
    data=data_yaml,
    epochs=epochs,
    imgsz=img_size,
    batch=batch_size,
    project=output_dir,
    name="train_experiment",
    exist_ok=True,
    # --- Mejoras de estabilidad ---
    lr0=lr_inicial,    # Tasa de aprendizaje ajustada según el origen
    patience=30,       # Si en 30 épocas no mejora, para solo (ahorra tiempo)
    save=True,
    pretrained=True    # Asegura que use los pesos cargados
)

# 4. Gestión de archivos finales
print("✅ Entrenamiento finalizado.")

# Definir la ruta donde YOLO dejó el mejor archivo
new_best_path = Path(output_dir) / "train_experiment" / "weights" / "best.pt"

if new_best_path.exists():
    # Aseguramos que la carpeta destino exista
    Path("/content/in").mkdir(parents=True, exist_ok=True)

    # Copiamos el nuevo modelo al destino
    shutil.copy2(new_best_path, best_pt_path)
    print(f"⭐ El nuevo 'best.pt' ha sido actualizado en: {best_pt_path}")
else:
    print("⚠️ Error: No se encontró el archivo best.pt generado.")

ℹ️ Iniciando desde cero con YOLOv8 Nano.
🚀 Iniciando entrenamiento con el dataset de 1300 imágenes...
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train_experiment, nbs=64, 

##
$$\textbf{Bloque 13 — Validación visual en imágenes (val):} \
\\\ \text{Usa best.pt para predecir sobre images/val y guarda imágenes con cajas dibujadas y las descarga.}$$


In [ ]:
# Genera predicciones sobre val y descarga un ZIP con los resultados (si falta algo, no falla: avisa)
run_name = "train"                                               # Nombre del entrenamiento (debe coincidir con Bloque 12)
pred_name = "predict_val"                                        # Nombre de carpeta de predicciones para val

best_path = DIR_RUNS/run_name/"weights/best.pt"                  # Ruta esperada del modelo best.pt
val_images_dir = DIR_DATASET/"images/val"                        # Ruta esperada del set de validación

# Revisa prerequisitos antes de predecir
if not best_path.exists():                                       # Si no existe best.pt
    print("⚠️ No encuentro best.pt todavía en:", best_path)      # Aviso
    print("ℹ️ Corre el Bloque 12 (entrenamiento) y asegúrate que termine bien.")  # Guía
elif not val_images_dir.exists():                                # Si no existe la carpeta val
    print("⚠️ No encuentro images/val en:", val_images_dir)      # Aviso
    print("ℹ️ Corre el Bloque 10 (normalización + split) para crear el dataset.")  # Guía
else:
    # Intenta ejecutar predicción sobre imágenes de validación
    try:
        print("✅ Iniciando predicción sobre val:")              # Mensaje de inicio
        print(" - Modelo:", best_path)                           # Muestra el modelo que se usará
        print(" - Fuente (val):", val_images_dir)                # Muestra la carpeta fuente
        print(" - Salida:", DIR_RUNS/pred_name)                  # Muestra la carpeta de salida

        model = YOLO(str(best_path))                             # Carga el modelo entrenado
        model.predict(                                           # Ejecuta predicción sobre val
            source=str(val_images_dir),                          # Fuente: imágenes val
            save=True,                                           # Guarda imágenes con cajas dibujadas
            project=str(DIR_RUNS),                               # Carpeta base de salida
            name=pred_name,                                      # Nombre de la carpeta de predicción
            exist_ok=True                                        # Reutiliza carpeta si ya existe
        )

        print("✅ Predicciones val guardadas en:", DIR_RUNS/pred_name)  # Confirmación final

    except Exception as e:
        print("⚠️ Falló la predicción en val por un error:")      # Aviso de error
        print("   ", str(e)[:400])                                # Muestra parte del error
        print("ℹ️ Tip típico: revisa rutas, nombres o si hay imágenes corruptas.")  # Guía rápida

    # Chequeo de outputs + creación de ZIP para descargar
    out_dir = DIR_RUNS/pred_name                                  # Carpeta de salida
    if not out_dir.exists():                                      # Si no existe salida
        print("⚠️ No existe la carpeta de salida:", out_dir)      # Aviso
    else:
        # Lista imágenes generadas
        out_imgs = sorted([p for p in out_dir.iterdir() if p.suffix.lower() in [".jpg",".png"]])  # Imágenes salida
        print("✅ Cantidad de imágenes generadas:", len(out_imgs)) # Cuenta
        if out_imgs:
            print("✅ Ejemplo:", out_imgs[0].name)                 # Muestra 1 ejemplo
        else:
            print("⚠️ No veo imágenes generadas en predict_val (puede ser que YOLO no haya guardado o que no haya detecciones).")  # Aviso

        # Crea ZIP descargable con todo el contenido de predict_val
        zip_path = DIR_OUT/"val_predictions.zip"                   # Ruta del zip final
        if zip_path.exists():                                      # Si ya existía
            zip_path.unlink()                                      # Borra el zip anterior

        with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:  # Crea zip
            for p in sorted(out_dir.rglob("*")):                   # Recorre todo dentro de predict_val
                if p.is_file():                                    # Solo archivos
                    z.write(p, arcname=str(p.relative_to(out_dir)))  # Agrega con ruta relativa

        print("✅ ZIP de validación creado en:", zip_path)          # Confirma zip

        # Descarga automática
        try:
            from google.colab import files                         # Importa descarga
            files.download(str(zip_path))                          # Descarga zip
            print("✅ Descarga iniciada: val_predictions.zip")      # Confirmación
        except Exception:
            print("⚠️ No pude iniciar descarga automática, pero el zip quedó en /content/out.")  # Aviso


✅ Iniciando predicción sobre val:
 - Modelo: /content/runs/train/weights/best.pt
 - Fuente (val): /content/dataset/images/val
 - Salida: /content/runs/predict_val

image 1/49 /content/dataset/images/val/000002.jpg: 384x640 (no detections), 46.6ms
image 2/49 /content/dataset/images/val/000007.jpg: 384x640 (no detections), 7.4ms
image 3/49 /content/dataset/images/val/000008.jpg: 384x640 (no detections), 7.0ms
image 4/49 /content/dataset/images/val/000009.jpg: 384x640 (no detections), 7.0ms
image 5/49 /content/dataset/images/val/000023.jpg: 384x640 2 Estrias, 7.1ms
image 6/49 /content/dataset/images/val/000024.jpg: 384x640 2 Estrias, 7.3ms
image 7/49 /content/dataset/images/val/000027.jpg: 384x640 2 Estrias, 9.1ms
image 8/49 /content/dataset/images/val/000029.jpg: 384x640 2 Estrias, 10.0ms
image 9/49 /content/dataset/images/val/000036.jpg: 384x640 2 Estrias, 7.0ms
image 10/49 /content/dataset/images/val/000040.jpg: 384x640 2 Estrias, 7.0ms
image 11/49 /content/dataset/images/val/000041.jp

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descarga iniciada: val_predictions.zip


#
$$\textbf{Modelo DAICH}$$

##
$$\textbf{Bloque 14 — Predicción en VIDEO (salida anotada + labels):} \
\\\ \text{Usa best.pt para predecir sobre un video y guarda: (1) video anotado y (2) archivos .txt por frame con detecciones (para luego armar CSVs).}$$


In [ ]:
# Corre predicción sobre un video usando best.pt y guarda resultados (si falta algo, no falla: avisa)
run_name = "train"                                                # Define el nombre del entrenamiento (igual que antes)
pred_video_name = "predict_video"                                 # Define el nombre de la carpeta de salida para video

best_path = DIR_RUNS/run_name/"weights/best.pt"                   # Ruta esperada del modelo entrenado best.pt
video_path = next((p for p in DIR_IN.iterdir() if p.suffix.lower() in [".mp4", ".mov", ".avi", ".mkv"]), None)  # Busca un video en /content/in

# Chequea prerequisitos sin tirar error
if not best_path.exists():                                        # Si no existe best.pt
    print("⚠️ No encuentro best.pt en:", best_path)               # Aviso
    print("ℹ️ Corre el Bloque 12 (entrenamiento) antes de este bloque.")  # Guía
elif video_path is None:                                          # Si no hay video subido
    print("⚠️ No detecto ningún video en /content/in.")           # Aviso
    print("ℹ️ Sube un video (mp4/mov/avi/mkv) y vuelve a correr este bloque.")  # Guía
else:
    # Intenta correr la predicción de YOLO sobre el video
    try:
        print("✅ Iniciando predicción en video:")                # Mensaje de inicio
        print(" - Modelo:", best_path)                            # Muestra el modelo usado
        print(" - Video:", video_path.name)                       # Muestra el video usado
        print(" - Salida:", DIR_RUNS/pred_video_name)             # Muestra la carpeta de salida

        model = YOLO(str(best_path))                              # Carga el modelo entrenado
        model.predict(                                            # Ejecuta predicción
            source=str(video_path),                               # Fuente: video
            save=True,                                            # Guarda el video/frames con cajas dibujadas
            save_txt=True,                                        # Guarda detecciones en .txt (por frame)
            save_conf=True,                                       # Incluye confidence en esos .txt
            project=str(DIR_RUNS),                                # Carpeta base de salida
            name=pred_video_name,                                 # Nombre del run de predicción
            exist_ok=True                                         # Reutiliza carpeta si ya existe
        )

        print("✅ Predicción completada en:", DIR_RUNS/pred_video_name)  # Confirmación

    except Exception as e:
        print("⚠️ Falló la predicción en video por un error:")    # Aviso de error
        print("   ", str(e)[:400])                                # Muestra parte del error
        print("ℹ️ Tip típico: revisa que el video no esté corrupto o que best.pt exista.")  # Guía

    # Chequea outputs esperados (sin romper)
    out_dir = DIR_RUNS/pred_video_name                            # Carpeta de salida del predict
    labels_dir = out_dir/"labels"                                 # Carpeta donde deberían quedar los .txt

    print("\n✅ Chequeo de outputs:")                              # Título del chequeo
    if out_dir.exists():                                          # Si existe la carpeta de salida
        print(" - Carpeta salida OK:", out_dir)                   # Confirma
    else:
        print("⚠️ No existe la carpeta de salida:", out_dir)      # Aviso

    if labels_dir.exists():                                       # Si existe la carpeta labels
        n_txt = len(list(labels_dir.glob("*.txt")))               # Cuenta cuantos txt hay
        print(" - Labels (.txt) OK:", labels_dir, "| cantidad:", n_txt)  # Confirma cantidad
        if n_txt == 0:                                            # Si no hay txt
            print("⚠️ labels/ existe pero no tiene .txt (puede ser que no detectó nada o falló el guardado).")  # Aviso
    else:
        print("⚠️ No encontré carpeta labels/ en:", labels_dir)   # Aviso
        print("ℹ️ Si no existe, revisa que predict se ejecutó con save_txt=True.")  # Guía

    # Intenta encontrar el video anotado generado y reportarlo
    annotated_candidates = []                                     # Lista para candidatos de video anotado
    for ext in [".mp4", ".mov", ".avi", ".mkv"]:                  # Revisa extensiones comunes
        annotated_candidates += list(out_dir.glob(f"*{ext}"))     # Agrega coincidencias

    if annotated_candidates:                                      # Si hay algún candidato
        annotated_video = annotated_candidates[0]                 # Toma el primero
        print(" - Video anotado detectado:", annotated_video.name)  # Confirma nombre
    else:
        print("⚠️ No pude detectar un video anotado en la carpeta de salida (a veces YOLO guarda frames en vez de video).")  # Aviso


NameError: name 'DIR_RUNS' is not defined

##
$$\textbf{Bloque 15 — Construir detections.csv (detecciones por frame/tiempo):} \
\\\ \text{Lee los .txt generados por YOLO en labels/ y crea una tabla (CSV) con frame, tiempo, clase, confianza y caja (x,y,w,h).}$$


In [ ]:
# Crea detections.csv desde los .txt de /content/runs/predict_video/labels (si falta algo, no falla: avisa)
import json                                                     # Sirve para leer salida JSON de ffprobe (fps del video)
import math                                                     # Utilidades matemáticas (por si se necesita)

pred_video_name = "predict_video"                               # Debe coincidir con el nombre usado en el Bloque 14
out_dir = DIR_RUNS/pred_video_name                              # Carpeta de salida de la predicción de video
labels_dir = out_dir/"labels"                                   # Carpeta donde YOLO deja los .txt por frame
det_csv_path = DIR_OUT/"detections.csv"                         # Ruta final del CSV de detecciones

# Intenta detectar el video en /content/in (para calcular tiempo real con fps)
video_path = next((p for p in DIR_IN.iterdir() if p.suffix.lower() in [".mp4", ".mov", ".avi", ".mkv"]), None)  # Busca video subido

# Función para obtener FPS real del video usando ffprobe (si falla, devuelve None)
def get_video_fps(path):                                        # Define función para leer fps del video
    try:
        cmd = f'ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of json "{path}"'  # Comando ffprobe
        r = subprocess.run(cmd, shell=True, capture_output=True, text=True)  # Ejecuta ffprobe
        data = json.loads(r.stdout)                              # Parsea JSON
        rate = data["streams"][0]["r_frame_rate"]                # Lee r_frame_rate (ej: "30000/1001")
        num, den = rate.split("/")                               # Separa numerador/denominador
        return float(num) / float(den)                           # Calcula fps como número
    except Exception:
        return None                                              # Si algo falla, retorna None

# Chequea prerequisitos sin romper el notebook
if not out_dir.exists():                                        # Si no existe carpeta de predicción de video
    print("⚠️ No existe la carpeta de predicción:", out_dir)     # Aviso
    print("ℹ️ Corre el Bloque 14 (predicción en video) antes de este bloque.")  # Guía
elif not labels_dir.exists():                                    # Si no existe labels/
    print("⚠️ No existe la carpeta labels/:", labels_dir)        # Aviso
    print("ℹ️ Asegúrate de correr el Bloque 14 con save_txt=True y save_conf=True.")  # Guía
else:
    # Obtiene fps del video (si no hay video o falla, usa fps=1 como fallback)
    fps = get_video_fps(video_path) if video_path else None      # Calcula fps real si hay video
    if fps is None:                                              # Si no se pudo obtener fps
        fps = 1.0                                                # Fallback para no romper el flujo
        print("⚠️ No pude obtener FPS del video; usaré fps=1.0 como aproximación para time_s.")  # Aviso
    else:
        print("✅ FPS detectado del video:", fps)                 # Confirma fps real

    # Lee todos los .txt de labels/ ordenados por nombre
    txt_files = sorted(labels_dir.glob("*.txt"))                 # Lista txt en labels/
    if len(txt_files) == 0:                                      # Si no hay txt
        print("⚠️ labels/ existe pero no tiene .txt. Puede que no haya detecciones o algo falló.")  # Aviso
    else:
        rows = []                                                # Lista para acumular filas del CSV

        # Recorre cada archivo .txt (cada uno representa un frame/imagen)
        for lf in txt_files:                                     # Recorre cada archivo de detección
            stem = lf.stem                                       # Nombre sin extensión (ej: "000001")
            digits = re.sub(r"\D", "", stem)                     # Extrae solo números del nombre
            if digits == "":                                     # Si no hay números, salta
                continue                                         # Evita errores de conversión

            frame_idx = int(digits)                              # Convierte a índice de frame
            time_s = frame_idx / fps                             # Convierte frame a tiempo en segundos (aprox)

            # Lee cada detección dentro del txt (una línea por detección)
            content = lf.read_text(encoding="utf-8").strip()     # Lee el contenido del archivo
            if content == "":                                    # Si está vacío, no hay detecciones en ese frame
                continue                                         # Salta

            for line in content.splitlines():                    # Recorre cada línea del txt
                parts = line.split()                             # Separa por espacios
                if len(parts) < 5:                               # Si no tiene lo mínimo YOLO
                    continue                                     # Salta líneas raras

                cls_id = int(parts[0])                           # ID de clase (0,1,2...)
                x = float(parts[1]); y = float(parts[2])         # Centro x,y normalizado (0-1)
                w = float(parts[3]); h = float(parts[4])         # Ancho/alto normalizado (0-1)

                # confidence solo existe si guardaste save_conf=True
                conf = float(parts[5]) if len(parts) >= 6 else None  # Lee confidence si existe

                # Obtiene nombre de clase si existe lista 'classes' (si no, deja el id)
                class_name = None                                # Inicializa nombre de clase
                if "classes" in globals() and isinstance(classes, list) and cls_id < len(classes):  # Si existe lista classes
                    class_name = classes[cls_id]                 # Traduce id a nombre
                else:
                    class_name = str(cls_id)                     # Fallback: usa el id como texto

                # Agrega una fila a la tabla
                rows.append({                                    # Crea un dict por detección
                    "frame": frame_idx,                          # Frame detectado
                    "time_s": time_s,                            # Tiempo aproximado en segundos
                    "class_id": cls_id,                          # ID de clase
                    "class_name": class_name,                    # Nombre de clase (si está definido)
                    "conf": conf,                                # Confianza (puede ser None)
                    "x": x, "y": y, "w": w, "h": h               # Caja en formato YOLO normalizado
                })

        # Convierte a DataFrame y guarda CSV
        det_df = pd.DataFrame(rows)                              # Crea tabla con todas las detecciones
        if det_df.empty:                                         # Si quedó vacío
            print("⚠️ No se generaron filas en detections.csv (quizás no hubo detecciones).")  # Aviso
        else:
            det_df = det_df.sort_values(["frame", "conf"], ascending=[True, False])  # Ordena por frame y mejor confianza
            det_df.to_csv(det_csv_path, index=False, encoding="utf-8")               # Guarda el CSV

            print("✅ detections.csv creado en:", det_csv_path)   # Confirma salida
            print("✅ Filas:", len(det_df), "| Frames únicos:", det_df["frame"].nunique())  # Resumen rápido
            print("✅ Ejemplo de 5 filas:")                       # Título ejemplo
            print(det_df.head(5).to_string(index=False))          # Muestra 5 filas


##
$$\textbf{Bloque 16 — Construir events.csv (intervalos/timeline):} \
\\\ \text{Convierte detections.csv en “eventos” por clase: agrupa frames cercanos en intervalos (inicio–fin) para evitar parpadeos y generar un timeline legible.}$$


In [ ]:
# Crea events.csv agrupando detecciones por clase en intervalos (si falta detections.csv, no falla: avisa)
events_csv_path = DIR_OUT/"events.csv"                             # Define dónde se guardará el archivo de eventos
det_csv_path = DIR_OUT/"detections.csv"                            # Ruta esperada del detections.csv

# Parámetros anti-parpadeo (ajustables)
min_conf = 0.35                                                     # Umbral mínimo de confianza para considerar detección
gap_frames = 2                                                      # Permite “huecos” de hasta N frames dentro de un mismo evento
min_len_frames = 3                                                  # Evento mínimo: requiere al menos N frames para existir

# Intenta obtener FPS del video para convertir frames a segundos (si falla, usa 1.0)
video_path = next((p for p in DIR_IN.iterdir() if p.suffix.lower() in [".mp4", ".mov", ".avi", ".mkv"]), None)  # Busca video en /content/in

def get_video_fps_safe(path):                                       # Función segura para obtener fps del video
    try:
        import json                                                 # Importa json para parsear ffprobe
        cmd = f'ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of json "{path}"'  # Comando ffprobe
        r = subprocess.run(cmd, shell=True, capture_output=True, text=True)  # Ejecuta ffprobe
        data = json.loads(r.stdout)                                 # Parsea JSON
        rate = data["streams"][0]["r_frame_rate"]                   # Lee r_frame_rate (ej: "30000/1001")
        num, den = rate.split("/")                                  # Separa numerador/denominador
        return float(num) / float(den)                              # Devuelve fps real
    except Exception:
        return 1.0                                                  # Fallback si falla

fps = get_video_fps_safe(video_path) if video_path else 1.0         # Calcula fps si hay video; si no, usa 1.0
print("✅ FPS usado para convertir frames→segundos:", fps)            # Confirma fps usado

# Si no existe detections.csv, avisa y termina sin romper el notebook
if not det_csv_path.exists():                                       # Verifica si existe detections.csv
    print("⚠️ No existe detections.csv en:", det_csv_path)          # Aviso
    print("ℹ️ Corre el Bloque 15 (detections.csv) antes de este bloque.")  # Guía
else:
    # Carga detections.csv (si está vacío o corrupto, avisa sin romper)
    try:
        det_df = pd.read_csv(det_csv_path)                          # Lee el CSV de detecciones
        print("✅ detections.csv cargado | filas:", len(det_df))     # Confirma carga
    except Exception as e:
        print("⚠️ No pude leer detections.csv por un error:")       # Aviso
        print("   ", str(e)[:300])                                   # Muestra parte del error
        det_df = pd.DataFrame()                                     # Deja DF vacío para no romper

    # Si no hay datos, avisa y termina
    if det_df.empty:                                                # Revisa si el DataFrame quedó vacío
        print("⚠️ detections.csv está vacío; no puedo generar events.csv.")  # Aviso
        print("ℹ️ Esto puede pasar si el modelo no detectó nada en el video.")  # Guía
    else:
        # Asegura columnas necesarias (si falta alguna, avisa y termina)
        needed = {"frame", "class_name", "conf"}                    # Columnas mínimas requeridas
        if not needed.issubset(set(det_df.columns)):                # Verifica columnas
            print("⚠️ detections.csv no tiene las columnas mínimas:", needed)  # Aviso
            print("ℹ️ Revisa que el Bloque 15 haya generado frame/class_name/conf.")  # Guía
        else:
            events_rows = []                                        # Lista donde guardaremos eventos (intervalos)

            # Recorre cada clase y crea intervalos de frames consecutivos (con tolerancia de gap)
            for cname in sorted(det_df["class_name"].dropna().unique()):  # Recorre clases detectadas
                sub = det_df[(det_df["class_name"] == cname)]       # Filtra por clase
                sub = sub[sub["conf"].fillna(0) >= min_conf]        # Filtra por confianza mínima (maneja NaN)

                frames = sorted(sub["frame"].dropna().astype(int).unique())  # Lista frames únicos ordenados
                if len(frames) == 0:                                # Si no hay frames para esta clase
                    continue                                        # Pasa a la siguiente clase

                start = frames[0]                                   # Inicio del evento actual
                prev = frames[0]                                    # Último frame visto del evento actual

                for fr in frames[1:]:                               # Recorre frames siguientes
                    if fr <= prev + 1 + gap_frames:                 # Si está cerca (con tolerancia de huecos)
                        prev = fr                                   # Extiende el evento actual
                    else:
                        # Cierra evento anterior si cumple largo mínimo
                        if (prev - start + 1) >= min_len_frames:    # Verifica largo mínimo del evento
                            events_rows.append([cname, start, prev])  # Guarda evento (clase, inicio, fin)
                        start = fr                                  # Abre nuevo evento
                        prev = fr                                   # Reinicia último frame

                # Cierra el último evento
                if (prev - start + 1) >= min_len_frames:            # Verifica largo mínimo final
                    events_rows.append([cname, start, prev])         # Guarda último evento

            # Si no hubo eventos (por filtros), avisa y termina sin romper
            if len(events_rows) == 0:                               # Revisa si se generó algo
                print("⚠️ No se generaron eventos con los parámetros actuales.")  # Aviso
                print("ℹ️ Prueba bajar min_conf o min_len_frames si te está quedando vacío.")  # Guía
            else:
                # Convierte eventos a DataFrame y calcula tiempos
                ev_df = pd.DataFrame(events_rows, columns=["class_name", "start_frame", "end_frame"])  # Crea tabla de eventos
                ev_df["start_s"] = ev_df["start_frame"] / fps       # Convierte inicio a segundos
                ev_df["end_s"] = ev_df["end_frame"] / fps           # Convierte fin a segundos
                ev_df["duration_s"] = (ev_df["end_s"] - ev_df["start_s"]).clip(lower=0)  # Calcula duración no negativa

                # Ordena eventos por tiempo de inicio
                ev_df = ev_df.sort_values(["start_s", "class_name"])  # Ordena cronológicamente

                # Guarda events.csv
                ev_df.to_csv(events_csv_path, index=False, encoding="utf-8")  # Guarda el CSV final

                # Prints de confirmación + resumen
                print("✅ events.csv creado en:", events_csv_path)   # Confirma salida
                print("✅ Eventos:", len(ev_df), "| Clases:", ev_df["class_name"].nunique())  # Resumen
                print("✅ Ejemplo de 10 eventos:")                   # Título ejemplo
                print(ev_df.head(10).to_string(index=False))         # Muestra 10 filas

                # Guarda los parámetros usados para trazabilidad
                params_path = DIR_OUT/"events_params.txt"            # Archivo para guardar parámetros
                params_text = (                                     # Texto de parámetros
                    f"min_conf={min_conf}\n"
                    f"gap_frames={gap_frames}\n"
                    f"min_len_frames={min_len_frames}\n"
                    f"fps_used={fps}\n"
                )
                params_path.write_text(params_text, encoding="utf-8")  # Escribe parámetros
                print("✅ Parámetros guardados en:", params_path)     # Confirma guardado


#
$$\textbf{Bloques auxiliares}$$

##
$$\textbf{Bloque 3 — Subida de archivos (cvat zip, videos, best.pt):} \
\\\ \text{Permite subir archivos desde tu computador a Colab y los deja guardados en content in para usarlos en el flujo.}$$

In [12]:
# Habilita el botón de subir archivos desde tu PC a Colab
from google.colab import files  # Herramienta de Colab para subir archivos manualmente

# Abre el selector para subir: video.mp4 o images.zip o cvat_yolo_export.zip
uploaded = files.upload()  # Te deja elegir archivos desde tu computador

# Mueve cada archivo subido a la carpeta estándar /content/in
for fname in uploaded.keys():              # Recorre los nombres de lo que subiste
    src = Path("/content")/fname           # Colab lo deja primero en /content
    dst = DIR_IN/fname                     # Destino final en /content/in
    if src.exists():                       # Confirma que el archivo está
        shutil.move(str(src), str(dst))    # Lo mueve a /content/in

# Muestra lo que quedó en /content/in para confirmar que está OK
print("✅ Archivos guardados en /content/in:")  # Mensaje de confirmación
items = list(DIR_IN.iterdir())                 # Lee el contenido de /content/in
if items:                                      # Si hay archivos
    for p in items:                            # Recorre cada archivo
        print(" -", p.name, f"({p.stat().st_size/1e6:.2f} MB)")  # Nombre y tamaño
else:
    print("⚠️ Carpeta vacía: no se subió nada aún.")             # Aviso si no hay archivos


Saving cvat t1p2 t10 y t3p2.zip to cvat t1p2 t10 y t3p2.zip
✅ Archivos guardados en /content/in:
 - cvat t1p2 t10 y t3p2.zip (28.30 MB)
 - .ipynb_checkpoints (0.00 MB)


##
$$\textbf{Bloque 6 — Video → Frames:} \
\\\ \text{Si hay un video, extrae imágenes (frames) a una frecuencia definida (fps) y las guarda en #/content/work/frames para etiquetarlas después.}$$


In [ ]:
# Busca un video por extensión común (si existe)
video = next((p for p in in_files if p.suffix.lower() in [".mp4", ".mov", ".avi", ".mkv"]), None)  # Encuentra el primer video

# Extrae frames desde el video detectado (si no hay video, no falla: solo avisa)
fps_extract = 1.0                                              # Define cuántas imágenes por segundo extraer (ej: 1.0 = 1 frame/seg)
img_ext = "jpg"                                                # Define el formato de imagen de salida (jpg o png)

# Si no hay video, no hacemos nada y seguimos
if video is None:                                              # Revisa si se detectó un video en /content/in
    print("⚠️ No hay video detectado en /content/in, así que no se extraen frames todavía.")  # Aviso sin romper el flujo
else:
    # Limpia frames anteriores para no mezclar corridas
    old_frames = list(FRAMES_DIR.glob(f"*.{img_ext}"))          # Busca frames antiguos en la carpeta de frames
    for f in old_frames:                                        # Recorre frames antiguos
        f.unlink()                                              # Borra cada frame antiguo

    # Define el número inicial desde el cual deseas comenzar la numeración
    start_number = 1  # Puedes poner el número que desees

    # Define el patrón de nombres con el número de inicio
    out_pattern = str(FRAMES_DIR / f"%06d.{img_ext}")

    # Construye y ejecuta el comando ffmpeg para extraer frames
    cmd = [
    "ffmpeg", "-y",
    "-i", str(video),
    "-vf", f"fps={fps_extract}",
    "-start_number", str(start_number), # <--- ESTO indica dónde empezar
    out_pattern]

    print("✅ Ejecutando FFmpeg para extraer frames:")          # Mensaje de inicio
    print("   ", " ".join(cmd))                                 # Muestra el comando para trazabilidad
    subprocess.run(cmd, check=False)                            # Ejecuta sin romper el notebook si ffmpeg devuelve error

    # Cuenta y muestra resultados
    frames = sorted(FRAMES_DIR.glob(f"*.{img_ext}"))            # Busca los frames generados
    if len(frames) == 0:                                        # Si no se generó ningún frame
        print("⚠️ No se generaron frames (revisa si el video está OK o si fps_extract es muy bajo).")  # Aviso
    else:
        print(f"✅ Frames listos: {len(frames)} en {FRAMES_DIR}")  # Confirma cuántos frames se crearon
        print("   Ejemplo primero/último:", frames[0].name, "|", frames[-1].name)  # Muestra nombres de ejemplo

# Comprime los frames en un ZIP descargable (si no hay frames, no falla: solo avisa)
zip_frames_path = DIR_OUT/"frames.zip"                         # Define dónde quedará el zip final

# Busca frames existentes en la carpeta de frames
frame_files = sorted(list(FRAMES_DIR.glob("*.jpg")) + list(FRAMES_DIR.glob("*.png")))  # Reúne frames jpg/png

# Si no hay frames aún, avisa y termina sin error
if len(frame_files) == 0:                                      # Revisa si hay frames para comprimir
    print("⚠️ No hay frames en /content/work/frames, así que no se puede crear frames.zip todavía.")  # Aviso
else:
    # Si ya existía un zip antiguo, lo borra para evitar confusiones
    if zip_frames_path.exists():                               # Verifica si el zip ya existe
        zip_frames_path.unlink()                               # Borra el zip anterior

    # Crea el zip con todos los frames
    with zipfile.ZipFile(zip_frames_path, "w", zipfile.ZIP_DEFLATED) as z:  # Abre un zip en modo escritura
        for f in frame_files:                                  # Recorre cada frame
            z.write(f, arcname=f.name)                         # Agrega el archivo al zip con su nombre

    # Confirma que el zip se creó correctamente
    print("✅ ZIP creado para CVAT:", zip_frames_path)          # Muestra la ruta del zip creado
    print(f"✅ Incluye {len(frame_files)} imágenes.")          # Indica cuántas imágenes quedaron dentro

    # Opción de descarga directa (si quieres)
    try:
        from google.colab import files                         # Importa herramienta de descarga de Colab
        files.download(str(zip_frames_path))                   # Descarga el zip a tu PC
        print("✅ Descarga iniciada (si tu navegador lo permite).")  # Confirmación
    except Exception:
        print("⚠️ No pude iniciar descarga automática, pero el zip quedó en /content/out para descargarlo manualmente.")  # Aviso



✅ Ejecutando FFmpeg para extraer frames:
    ffmpeg -y -i /content/in/TRAMO 3 - PARTE 2.mp4 -vf fps=1.0 -start_number 590 /content/work/frames/%06d.jpg
✅ Frames listos: 296 en /content/work/frames
   Ejemplo primero/último: 000590.jpg | 000885.jpg
✅ ZIP creado para CVAT: /content/out/frames.zip
✅ Incluye 296 imágenes.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descarga iniciada (si tu navegador lo permite).


##
$$\textbf{Bloque 17 — Empaquetar resultados y descargar best pt u otros} \
\\\ \text{Copia best.pt y los CSV (detections/events) a #/content/out, busca el video anotado si existe, arma un ZIP final y lo descarga.}$$


In [ ]:
# Empaqueta outputs finales en /content/out y crea final_report.zip (si falta algo, no falla: avisa)
run_name = "train"                                                 # Nombre del entrenamiento usado
pred_video_name = "predict_video"                                  # Nombre de carpeta de predicción de video

best_path = DIR_RUNS/run_name/"weights/best.pt"                    # Ruta esperada del best.pt
det_csv_path = DIR_OUT/"detections.csv"                            # Ruta esperada del detections.csv
events_csv_path = DIR_OUT/"events.csv"                             # Ruta esperada del events.csv

final_zip_path = DIR_OUT/"final_report.zip"                        # Ruta del zip final
best_out_path = DIR_OUT/"best.pt"                                  # Copia final de best.pt en /out

pred_dir = DIR_RUNS/pred_video_name                                # Carpeta donde quedaron outputs del predict video
annotated_out_path = DIR_OUT/"annotated.mp4"                       # Nombre estándar del video anotado en /out

# Lista de archivos que intentaremos meter al zip
to_zip = []                                                        # Lista de rutas para incluir en el zip

print("✅ Preparando empaquetado de resultados...")                 # Mensaje de inicio

# Copia best.pt a /out si existe
if best_path.exists():                                             # Si existe best.pt
    shutil.copy2(best_path, best_out_path)                          # Copia best.pt a /content/out
    to_zip.append(best_out_path)                                   # Agrega best.pt a lista para zip
    print("✅ best.pt listo en:", best_out_path)                    # Confirma
else:
    print("⚠️ No encontré best.pt en:", best_path)                  # Aviso

# Agrega detections.csv si existe
if det_csv_path.exists():                                          # Si existe detections.csv
    to_zip.append(det_csv_path)                                    # Agrega al zip
    print("✅ detections.csv listo en:", det_csv_path)              # Confirma
else:
    print("⚠️ No encontré detections.csv en:", det_csv_path)        # Aviso

# Agrega events.csv si existe
if events_csv_path.exists():                                       # Si existe events.csv
    to_zip.append(events_csv_path)                                 # Agrega al zip
    print("✅ events.csv listo en:", events_csv_path)               # Confirma
else:
    print("⚠️ No encontré events.csv en:", events_csv_path)         # Aviso

# Intenta detectar un video anotado en la carpeta de predicción
annotated_video = None                                             # Variable para guardar ruta del video anotado
if pred_dir.exists():                                              # Si existe carpeta predict_video
    candidates = []                                                # Lista de posibles videos generados
    for ext in [".mp4", ".mov", ".avi", ".mkv"]:                   # Extensiones comunes de video
        candidates += list(pred_dir.glob(f"*{ext}"))               # Busca archivos de video en la carpeta
    if candidates:                                                 # Si encontró alguno
        annotated_video = candidates[0]                            # Toma el primero
        shutil.copy2(annotated_video, annotated_out_path)          # Copia a /content/out con nombre estándar
        to_zip.append(annotated_out_path)                          # Agrega al zip
        print("✅ Video anotado listo en:", annotated_out_path)     # Confirma
    else:
        print("⚠️ No encontré video anotado en:", pred_dir)         # Aviso
else:
    print("⚠️ No existe la carpeta de predicción de video:", pred_dir)  # Aviso

# Si no hay nada para comprimir, avisa y termina sin error
if len(to_zip) == 0:                                               # Revisa si hay archivos en lista
    print("⚠️ No hay archivos suficientes para crear el zip final todavía.")  # Aviso
    print("ℹ️ Corre entrenamiento/predicción y generación de CSV antes de empaquetar.")  # Guía
else:
    # Borra zip anterior si existía para evitar confusión
    if final_zip_path.exists():                                    # Si ya existía final_report.zip
        final_zip_path.unlink()                                    # Lo borra

    # Crea el zip final con todo lo disponible
    with zipfile.ZipFile(final_zip_path, "w", zipfile.ZIP_DEFLATED) as z:  # Abre zip para escribir
        for p in to_zip:                                           # Recorre archivos a incluir
            z.write(p, arcname=p.name)                              # Agrega con nombre simple dentro del zip

    # Confirma creación del zip final
    print("✅ ZIP final creado:", final_zip_path)                   # Confirma ruta del zip
    print("✅ Incluye:", [p.name for p in to_zip])                  # Muestra qué incluyó

    # Intenta descargar automáticamente el zip
    try:
        from google.colab import files                              # Importa herramienta de descarga
        files.download(str(final_zip_path))                         # Descarga zip
        print("✅ Descarga iniciada (si tu navegador lo permite).")  # Confirmación
    except Exception:
        print("⚠️ No pude iniciar descarga automática, pero el zip quedó en /content/out.")  # Aviso


✅ Preparando empaquetado de resultados...
✅ best.pt listo en: /content/out/best.pt
⚠️ No encontré detections.csv en: /content/out/detections.csv
⚠️ No encontré events.csv en: /content/out/events.csv
⚠️ No existe la carpeta de predicción de video: /content/runs/predict_video
✅ ZIP final creado: /content/out/final_report.zip
✅ Incluye: ['best.pt']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descarga iniciada (si tu navegador lo permite).


##
$$\textbf{Bloque 19 — Interfaz simple (modo “usuario final”):} \
\\\ \text{Permite subir un modelo best.pt y un video, ejecuta la predicción y genera final\_report.zip (video anotado + CSVs) en #/content/out.}$$


In [ ]:
# Interfaz mínima: subes best.pt + video y el notebook te genera el zip final (si falta algo, no falla: avisa)
from google.colab import files                                  # Herramienta de Colab para subir/descargar archivos

# Define nombres estándar de trabajo
UI_MODEL_NAME = "best.pt"                                       # Nombre esperado para el modelo subido
UI_VIDEO_NAME = None                                            # Nombre del video subido (lo detectamos por extensión)

# Crea carpetas por si no existen (por seguridad)
for d in [DIR_IN, DIR_RUNS, DIR_OUT]:                           # Asegura carpetas clave
    d.mkdir(parents=True, exist_ok=True)                        # Crea si falta

print("✅ Interfaz simple: sube best.pt y un video (mp4/mov/avi/mkv).")  # Instrucción amigable

# Subida de archivos (el usuario elige desde su PC)
uploaded = files.upload()                                       # Abre selector para subir archivos

# Mueve lo subido a /content/in para mantener el estándar
for fname in uploaded.keys():                                   # Recorre nombres de archivos subidos
    src = Path("/content")/fname                                # Ruta temporal donde Colab deja los archivos
    dst = DIR_IN/fname                                          # Ruta final en /content/in
    if src.exists():                                            # Verifica existencia
        shutil.move(str(src), str(dst))                         # Mueve a /content/in

# Detecta best.pt y video dentro de /content/in
model_path = DIR_IN/UI_MODEL_NAME                               # Ruta esperada del modelo (best.pt)
video_path = next((p for p in DIR_IN.iterdir() if p.suffix.lower() in [".mp4",".mov",".avi",".mkv"]), None)  # Busca video

# Valida inputs sin romper
if not model_path.exists():                                     # Si no existe el modelo subido
    print("⚠️ No encontré best.pt en /content/in.")             # Aviso
    print("ℹ️ Sube un archivo llamado exactamente 'best.pt'.")  # Guía
elif video_path is None:                                        # Si no se detectó video
    print("⚠️ No encontré un video en /content/in.")            # Aviso
    print("ℹ️ Sube un video .mp4/.mov/.avi/.mkv junto al best.pt.")  # Guía
else:
    # Define carpeta de salida para esta predicción
    pred_video_name = "predict_video_ui"                        # Nombre del run de predicción para interfaz
    out_dir = DIR_RUNS/pred_video_name                          # Carpeta donde YOLO guardará resultados
    labels_dir = out_dir/"labels"                               # Carpeta donde quedarán txt por frame

    # Ejecuta predicción en video y guarda video + txt + conf
    try:
        print("✅ Ejecutando predicción (interfaz):")            # Mensaje de inicio
        print(" - Modelo:", model_path)                          # Muestra modelo
        print(" - Video:", video_path.name)                      # Muestra video
        print(" - Salida:", out_dir)                             # Muestra salida

        model = YOLO(str(model_path))                            # Carga el modelo subido
        model.predict(                                           # Ejecuta predicción
            source=str(video_path),                               # Video de entrada
            save=True,                                            # Guarda salida visual
            save_txt=True,                                        # Guarda txt por frame
            save_conf=True,                                       # Guarda confidence
            project=str(DIR_RUNS),                                # Carpeta base
            name=pred_video_name,                                 # Nombre del run
            exist_ok=True                                         # Reutiliza si existe
        )

        print("✅ Predicción terminada.")                         # Confirmación
    except Exception as e:
        print("⚠️ Falló la predicción por un error:")            # Aviso de error
        print("   ", str(e)[:400])                               # Muestra parte del error
        print("ℹ️ Revisa que best.pt sea compatible con tu dataset/clases.")  # Guía

    # Construye detections.csv si existen labels
    det_csv_path = DIR_OUT/"detections.csv"                      # Ruta detections final
    events_csv_path = DIR_OUT/"events.csv"                       # Ruta events final

    if not labels_dir.exists():                                  # Si no existe labels/
        print("⚠️ No existe labels/ en la salida, así que no puedo construir CSVs.")  # Aviso
    else:
        # Obtiene fps real para time_s (si falla, usa 1.0)
        def get_video_fps_safe(path):                            # Función segura para fps
            try:
                import json                                      # Para parsear JSON
                cmd = f'ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of json "{path}"'  # ffprobe
                r = subprocess.run(cmd, shell=True, capture_output=True, text=True)  # Ejecuta
                data = json.loads(r.stdout)                      # Parsea
                rate = data["streams"][0]["r_frame_rate"]        # Lee rate
                num, den = rate.split("/")                       # Separa
                return float(num)/float(den)                     # Calcula fps
            except Exception:
                return 1.0                                       # Fallback

        fps = get_video_fps_safe(video_path)                     # FPS para convertir frames a segundos
        print("✅ FPS usado:", fps)                               # Confirma fps

        # Lee txt y arma detections.csv
        txt_files = sorted(labels_dir.glob("*.txt"))             # Lista txt por frame
        rows = []                                                # Filas para detections

        for lf in txt_files:                                     # Recorre cada txt
            digits = re.sub(r"\D", "", lf.stem)                  # Extrae frame desde el nombre
            if digits == "":                                     # Si no hay número
                continue                                         # Salta
            frame_idx = int(digits)                              # Frame
            time_s = frame_idx / fps                             # Tiempo en segundos (aprox)
            content = lf.read_text(encoding="utf-8").strip()     # Lee contenido
            if content == "":                                    # Si vacío
                continue                                         # Salta

            for line in content.splitlines():                    # Recorre detecciones
                parts = line.split()                             # Separa
                if len(parts) < 5:                               # Si no cumple mínimo
                    continue                                     # Salta
                cls_id = int(parts[0])                           # Clase id
                x, y, w, h = map(float, parts[1:5])              # Caja
                conf = float(parts[5]) if len(parts) >= 6 else None  # Conf si existe
                rows.append({"frame": frame_idx, "time_s": time_s, "class_id": cls_id, "conf": conf,
                             "x": x, "y": y, "w": w, "h": h})    # Agrega fila

        det_df = pd.DataFrame(rows)                              # Crea DataFrame
        if det_df.empty:                                         # Si no hay detecciones
            print("⚠️ No hubo detecciones, detections.csv quedará vacío/no se generará events.csv.")  # Aviso
        else:
            det_df = det_df.sort_values(["frame","conf"], ascending=[True, False])  # Ordena
            det_df.to_csv(det_csv_path, index=False, encoding="utf-8")              # Guarda CSV
            print("✅ detections.csv listo en:", det_csv_path)    # Confirma

            # Genera events.csv agrupando por class_id (interfaz simple, sin nombres)
            min_conf = 0.35                                      # Umbral conf
            gap_frames = 2                                       # Huecos tolerados
            min_len_frames = 3                                   # Largo mínimo

            events_rows = []                                     # Lista eventos
            for cid in sorted(det_df["class_id"].unique()):      # Recorre clases
                sub = det_df[(det_df["class_id"] == cid) & (det_df["conf"].fillna(0) >= min_conf)]  # Filtra
                frames = sorted(sub["frame"].unique())           # Frames
                if not frames:                                   # Si vacío
                    continue                                     # Salta
                start = frames[0]; prev = frames[0]              # Inicializa evento
                for fr in frames[1:]:                            # Recorre frames
                    if fr <= prev + 1 + gap_frames:              # Si cercano
                        prev = fr                                # Extiende
                    else:
                        if (prev - start + 1) >= min_len_frames: # Si cumple mínimo
                            events_rows.append([cid, start, prev])  # Guarda
                        start = fr; prev = fr                    # Nuevo evento
                if (prev - start + 1) >= min_len_frames:         # Cierra último
                    events_rows.append([cid, start, prev])        # Guarda

            ev_df = pd.DataFrame(events_rows, columns=["class_id","start_frame","end_frame"])  # Tabla eventos
            if ev_df.empty:                                      # Si no hay eventos
                print("⚠️ No se generaron eventos con los parámetros actuales.")  # Aviso
            else:
                ev_df["start_s"] = ev_df["start_frame"]/fps      # Inicio s
                ev_df["end_s"] = ev_df["end_frame"]/fps          # Fin s
                ev_df["duration_s"] = (ev_df["end_s"]-ev_df["start_s"]).clip(lower=0)  # Duración
                ev_df.to_csv(events_csv_path, index=False, encoding="utf-8")            # Guarda CSV
                print("✅ events.csv listo en:", events_csv_path)  # Confirma

    # Copia video anotado (si existe) y arma zip final
    final_zip_path = DIR_OUT/"final_report.zip"                  # Ruta zip final
    to_zip = []                                                  # Lista archivos para zip

    if model_path.exists():                                      # Si modelo existe
        to_zip.append(model_path)                                # Agrega best.pt
    if det_csv_path.exists():                                    # Si detections existe
        to_zip.append(det_csv_path)                              # Agrega detections
    if events_csv_path.exists():                                 # Si events existe
        to_zip.append(events_csv_path)                           # Agrega events

    # Busca un video anotado en la carpeta de salida y lo copia a /out
    annotated_candidates = []                                    # Lista candidatos
    for ext in [".mp4",".mov",".avi",".mkv"]:                    # Exts video
        annotated_candidates += list(out_dir.glob(f"*{ext}"))     # Busca videos
    if annotated_candidates:                                     # Si encontró
        shutil.copy2(annotated_candidates[0], DIR_OUT/"annotated.mp4")  # Copia a /out
        to_zip.append(DIR_OUT/"annotated.mp4")                   # Agrega al zip
        print("✅ annotated.mp4 listo en /content/out.")          # Confirma
    else:
        print("⚠️ No encontré video anotado en la salida.")       # Aviso

    # Crea zip final si hay algo para comprimir
    if len(to_zip) == 0:                                         # Si no hay nada
        print("⚠️ No hay archivos para empaquetar todavía.")      # Aviso
    else:
        if final_zip_path.exists():                              # Si zip existe
            final_zip_path.unlink()                              # Lo borra
        with zipfile.ZipFile(final_zip_path, "w", zipfile.ZIP_DEFLATED) as z:  # Crea zip
            for p in to_zip:                                     # Recorre archivos
                z.write(p, arcname=p.name)                       # Agrega al zip
        print("✅ final_report.zip creado en:", final_zip_path)   # Confirma

        # Descarga zip final
        try:
            files.download(str(final_zip_path))                  # Descarga zip
            print("✅ Descarga iniciada.")                        # Confirmación
        except Exception:
            print("⚠️ No pude iniciar descarga automática, pero quedó en /content/out.")  # Aviso
